<a href="https://colab.research.google.com/github/lakshitgosain/Transformers--Implementation--Andrej-kaparthy/blob/main/Transformers_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

GPT - Generative Pretrained Transformers
- It is a base from Attention is all you need
- In this tutorial, we are not going to recreate what's there for ChatGPT as it's a very high level, production grade system that involves training on whole of the internet's data(most of not whole) and involves rigorous pretraining methods

In [1]:
#We will use tiny shakespere

We are going to create a character level transformer here and it's going to predict the next character.
- In chatgpt, the output is based of token level prediction. It predicts next token(word) and not the next character

https://github.com/karpathy/nanoGPT - Repo for training transformers on any given text .
We are defining/creating the nonogpt from scratch

In [2]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-01-02 08:20:50--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.05s   

2024-01-02 08:20:50 (20.8 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [3]:
with open("input.txt",'r') as f:
  text=f.read()

In [4]:
print("length of the text is :", len(text))

length of the text is : 1115394


In [5]:
#Let's look at the first 1000 characters
print("First 1000 characters", text[:1000])

First 1000 characters First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in

In [6]:
chars=(sorted(list(set(text))))
vocab_size=len(chars)
print("".join(chars)) #All the characters that have been used in the dataset
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65




```
# This is formatted as code
```

Strategy ome strategy to tokenize the text. We will be translating characters to integers.
Google uses SentencePiece.
OpenAI uses Tiktoken. THis has 50000 tokens. We in our use case have 65 tokens as we saw above.


In [12]:
stoi={ch:i for i,ch in enumerate(chars)}
itos={i:ch for i,ch in enumerate(chars)}
encode= lambda s: [stoi[c] for c in s] # Encoder: takes a stering and outputs a list of integers
decode= lambda l: "".join([itos[i] for i in l])

In [13]:
encode("Hello There")

[20, 43, 50, 50, 53, 1, 32, 46, 43, 56, 43]

In [15]:
import torch

In [16]:
data=torch.tensor(encode(text),dtype=torch.long)
print(data.shape,data.dtype)
print(data[:1000]) #The 1000 characters we looked at earlier will be the GPT looks like this

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [18]:
#The whole dataset is now represented as a very large sequence of integers above

In [19]:
n=int(0.9*len(data))
train_data=data[:n]
val_data=data[n:]

In [21]:
#We will now start plugging the text into the transformers model for it to learn. We will do it in chunks and train chunks at a time.

In [23]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [26]:
x=train_data[:block_size]
y=train_data[1:block_size+1]
for t in range(block_size):
  context=x[:t+1]
  target=y[t]
  print(f"when the input is {context} the target is {target}")
  #Look at the output of the for loop. this is the reason why we offset to +1 as we need the output of the tensor also present
  #We train on the block size nopt just for efficiency but also to make the transformer to see the context from all the way to 1 to block size and see everythin in between.
  #Transformer knows how to predict everything upto block size and know how to predict

when the input is tensor([18]) the target is 47
when the input is tensor([18, 47]) the target is 56
when the input is tensor([18, 47, 56]) the target is 57
when the input is tensor([18, 47, 56, 57]) the target is 58
when the input is tensor([18, 47, 56, 57, 58]) the target is 1
when the input is tensor([18, 47, 56, 57, 58,  1]) the target is 15
when the input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is 47
when the input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is 58


In [27]:
#We will have batches of the blocks/chunks of characters for efficiency so that we keep the CPU/GPU busy and train parallely on those batches


In [41]:
torch.manual_seed(1337)
batch_size=4 #How many independent sequences will be processed in parallel
block_size=8 # What is the maximum context length of predictions

def get_batch(split):
  data= train_data if split=="train" else val_data
  ix=torch.randint(len(data) - block_size, (batch_size,)) #offsets into the trainiing set
  print("ix",ix)
  x=torch.stack([data[i:i+block_size] for i in ix]) #Concatenates a sequence of tensors along a new dimension (dim), all tensors must be same size.
  y=torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x,y

xb,yb= get_batch("train")
print("inputs:")
print(xb.shape)
print(xb)
print("Targets: ")
print(yb.shape)
print(yb)

print("------")

for b in range(batch_size): #Batch Dimension
  for t in range(block_size): #Time dimension
    context=xb[b, :t+1]
    target=yb[b,t]
    print(f"when input is {context.tolist()} the target is {target}")


ix tensor([ 76049, 234249, 934904, 560986])
inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
Targets: 
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
------
when input is [24] the target is 43
when input is [24, 43] the target is 58
when input is [24, 43, 58] the target is 5
when input is [24, 43, 58, 5] the target is 57
when input is [24, 43, 58, 5, 57] the target is 1
when input is [24, 43, 58, 5, 57, 1] the target is 46
when input is [24, 43, 58, 5, 57, 1, 46] the target is 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target is 39
when input is [44] the target is 53
when input is [44, 53] the target is 56
when input is [44, 53, 56] the target is 1
when input is [44, 53, 56, 1] the target is 58

In [42]:
#The above contains 32 examples. When the input 1 token which is being predicted.


In [43]:
#We now have a batch of input we need to input to the transformer model.


In [ ]:
#We will use the bigram language model
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)